In [ ]:
import numpy as np
import os

# Set various TF options
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
import keras.api as keras

tf.debugging.set_log_device_placement(False)

print("TF version:", tf.__version__)
print("TF device:", tf.config.list_physical_devices('GPU')) # or 'CPU'

In [ ]:
from common import CV_DATA_DIR
from pathlib import Path

DATASET_PATH = Path(CV_DATA_DIR, 'homer_bart_2.zip')

archive = keras.utils.get_file(origin=DATASET_PATH.as_uri(), extract=True)
data_dir = Path(archive).with_suffix('')

In [ ]:
image_shape = (128, 128, 3)

train_ds = keras.utils.image_dataset_from_directory(
    Path(data_dir, 'training_set'),
    image_size=image_shape[:2],
    batch_size=8,
    labels='inferred'
)
val_ds = keras.utils.image_dataset_from_directory(
    Path(data_dir, 'test_set'),
    image_size=image_shape[:2],
    batch_size=1,
    labels='inferred',
    shuffle=False
)

In [ ]:
from keras.api import layers, models

hidden_layer_neurons = int((1152 + 2) / 2)
num_classes = 2

model = models.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(units=hidden_layer_neurons, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=30
)

In [ ]:
predictions = model.predict(val_ds)
predictions = np.argmax(predictions, axis=1)

In [ ]:
classes = []
for image, clazz in val_ds.as_numpy_iterator():
    classes.append(clazz)
classes = np.asarray(classes)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(classes, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(classes, predictions))